### Import librerie ed oggetti

* pandas, per la manipolazione dei dati csv
* da sklearn:
    * DecisionTreeClassifier
    * LabelEncoder, per gestire variabili categoriali
    * KNeighborClassifier per utilizzare il classificatore KNN
    * RandomForestClassifier (da Kaggle)
    * SVM
    * Naive Bayes
    
Mi sono divertita a provare l'accuratezza usando diversi metodi, visti anche nel corso di Machine Learning.

In [47]:
!sudo apt update && sudo apt install -y graphviz

/bin/bash: sudo: command not found


In [92]:
!pip3 install graphviz
!pip3 install mglearn
!pip3 install seaborn
!pip3 install scikit-learn=='0.24.1'

     |████████████████████████████████| 285 kB 17.7 MB/s eta 0:00:01


In [116]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

### Importo i file csv

In [117]:
telco_train = pd.read_csv('train.csv', error_bad_lines=False,dtype=str)
telco_test = pd.read_csv('test.csv', error_bad_lines=False,dtype=str)

### Preprocessing dei dati

Prepariamo i dati:
* Per prima cosa visualizziamo i dati.
* Controllo che non ci siano duplicati o valori nulli.
* Effettuo l'encoding delle stringhe in interi.
* Cerco le feature più significative.
* Elimino le feature che hanno meno influenza sul risultato.

In [119]:
#visualizzo i dati
telco_train

,Churn,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,No,2040-LDIWQ,Male,0,Yes,Yes,65,Yes,Yes,DSL,...,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),84.2,5324.5
1,No,0458-HEUZG,Female,0,No,No,13,No,No phone service,DSL,...,No,Yes,Yes,No,No,Two year,No,Mailed check,35.4,450.4
2,No,2587-YNLES,Female,0,Yes,Yes,6,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.1,100.35
3,No,3865-YIOTT,Male,0,Yes,Yes,72,Yes,Yes,Fiber optic,...,No,Yes,No,Yes,Yes,One year,Yes,Bank transfer (automatic),106.1,7848.5
4,No,2144-BFDSO,Female,1,Yes,No,50,Yes,Yes,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,No,Bank transfer (automatic),108.75,5431.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4502,No,5446-DKWYW,Female,1,Yes,Yes,61,Yes,Yes,DSL,...,No,Yes,No,Yes,No,One year,Yes,Electronic check,65.2,3965.05
4503,Yes,5832-XKAES,Male,0,No,No,2,Yes,No,Fiber optic,...,No,No,No,No,No,Month-to-month,Yes,Credit card (automatic),69.8,134.7
4504,Yes,9692-TUSXH,Female,0,No,No,18,Yes,No,Fiber optic,...,No,No,No,Yes,Yes,Month-to-month,Yes,Electronic check,96.05,1740.7
4505,Yes,8387-MOJJT,Female,0,Yes,Yes,42,Yes,No,Fiber optic,...,No,No,No,Yes,No,Month-to-month,Yes,Credit card (automatic),77.95,3384


In [118]:
#controllo che non ci siano duplicati
telco_train.drop_duplicates(inplace=True)
telco_test.drop_duplicates(inplace=True)

In [120]:
#Effettua l'encoding delle stringhe in interi
label_encoder = LabelEncoder()
telco_train_encoded = telco_train.apply(label_encoder.fit_transform)
telco_test_encoded = telco_test.apply(label_encoder.fit_transform)

In [121]:
#controllo che non ci siano valori nullli 
telco_train.isnull().any()
telco_test.isnull().any()

Churn               False
customerID          False
gender              False
SeniorCitizen       False
Partner             False
Dependents          False
tenure              False
PhoneService        False
MultipleLines       False
InternetService     False
OnlineSecurity      False
OnlineBackup        False
DeviceProtection    False
TechSupport         False
StreamingTV         False
StreamingMovies     False
Contract            False
PaperlessBilling    False
PaymentMethod       False
MonthlyCharges      False
TotalCharges        False
dtype: bool

In [122]:
#visualizza la corrrelazione tra le features
print(telco_train_encoded.corr())

#positivo: SeniorCitizen, PhoneService, MultipleLines, PaperlessBilling, PaymentMethod, MonthlyCharges, TotalCharges
#negativo: customerID, gender, Partner, Dependets, tenure, InternetService, OnlineBackup, OnlineSecurity, DeviceProtection, TechSupport, StreamingTV, StreamingMovies, Contract 

                     Churn  customerID    gender  SeniorCitizen   Partner  \
Churn             1.000000   -0.035364 -0.005766       0.150079 -0.148929   
customerID       -0.035364    1.000000  0.004930      -0.003350 -0.029233   
gender           -0.005766    0.004930  1.000000       0.011912 -0.018644   
SeniorCitizen     0.150079   -0.003350  0.011912       1.000000  0.018099   
Partner          -0.148929   -0.029233 -0.018644       0.018099  1.000000   
Dependents       -0.167739   -0.026389 -0.002372      -0.220322  0.441371   
tenure           -0.245804    0.008111 -0.013502       0.007382  0.285546   
PhoneService      0.011616   -0.031296 -0.014099       0.013170  0.041815   
MultipleLines     0.040933    0.014753 -0.006313       0.126672  0.141346   
InternetService  -0.042126   -0.028573 -0.007434      -0.027423  0.014566   
OnlineSecurity   -0.284853    0.026172 -0.026776      -0.132201  0.157156   
OnlineBackup     -0.190973    0.002691 -0.027497      -0.019539  0.157362   

In [124]:
#Una delle features con valore più significativo è Contract. Mi salvo i tipi di contratti, 
# e vedo se c'è qualche correlazione con il valore di Churn.

tipi = telco_train['Contract'].unique()
n_per_tipo = []
for tipo in tipi: 
    n_per_tipo.append([tipo, telco_train.Contract.eq(tipo).sum(), telco_train[(telco_train.Contract == tipo) & (telco_train.Churn== 'Yes')].shape[0]])
print(n_per_tipo)

#il maggior numero di utenti con valore Churn == yes sono quelli con contratto mese per mese.

[['Two year', 1078, 27], ['One year', 949, 108], ['Month-to-month', 2480, 1054]]


In [125]:
#Un'altra feature con valore significativo rispetto a Churn è OnlineSecurity.
tipi2= telco_train['OnlineSecurity'].unique()
n_per_tipo2 = []
for tipo in tipi2: 
    n_per_tipo2.append([tipo, telco_train.OnlineSecurity.eq(tipo).sum(), telco_train[(telco_train.OnlineSecurity == tipo) & (telco_train.Churn== 'Yes')].shape[0]])
print(n_per_tipo2)

#Il maggior numero di utenti con Churn==yes fanno parte degli utenti con OnlineSecurity==No

[['No', 2252, 927], ['No internet service', 955, 74], ['Yes', 1300, 188]]


In [126]:
#Un'altra feature con valore significativo rispetto a Churn è TechSupport.
tipi3= telco_train['TechSupport'].unique()
n_per_tipo3 = []
for tipo in tipi3: 
    n_per_tipo3.append([tipo, telco_train.TechSupport.eq(tipo).sum(), telco_train[(telco_train.TechSupport == tipo) & (telco_train.Churn== 'Yes')].shape[0]])
print(n_per_tipo3)

#Il maggior numero di utenti con Churn==yes fanno parte degli utenti con TechSecurity==No

[['Yes', 1302, 198], ['No internet service', 955, 74], ['No', 2250, 917]]


In [127]:
#Notiamo che le feature che hanno minore impatto sul valore di Churn sono customerID, gender, PhoneService
#MultipleLines, InternetService, StreamingTV, StreamingMovies, TotalCharges. Le elimino.

telco_train_encoded.drop(["customerID", "gender", "PhoneService", "MultipleLines", "InternetService", "StreamingTV", "StreamingMovies", "TotalCharges"], axis = 1, inplace = True)
telco_train.drop(["customerID", "gender", "PhoneService", "MultipleLines", "InternetService", "StreamingTV", "StreamingMovies", "TotalCharges"], axis = 1, inplace = True)

telco_test_encoded.drop(["customerID", "gender", "PhoneService", "MultipleLines", "InternetService", "StreamingTV", "StreamingMovies", "TotalCharges"], axis = 1, inplace = True)
telco_test.drop(["customerID", "gender", "PhoneService", "MultipleLines", "InternetService", "StreamingTV", "StreamingMovies", "TotalCharges"], axis = 1, inplace = True)

In [128]:
#splitta i dati di train per avere feature (X_train) e target (y_train)
y_train = telco_train_encoded.pop('Churn')
X_train = telco_train_encoded

#splitta i dati di test per avere feature (X_test) e target (y_train)
y_test = telco_test_encoded.pop('Churn')
X_test = telco_test_encoded

### Classificazione

Ho provato diversi classificatori. La logistic regression restituisce l'accuratezza migliore.

In [129]:
# effettuo la classificazione con KNN
knc = KNeighborsClassifier(n_neighbors=1)
knc.fit(X_train, y_train)
print("Test set accuracy with KNN: {:.3f}".format(knc.score(X_test, y_test)))

Test set accuracy with KNN: 0.643


In [130]:
# effettuo la classificazione con random forest classifier 
rand_for = RandomForestClassifier(random_state=0)
rand_for.fit(X_train, y_train)
print("Test set accuracy with Random Forest Classifier: {:.3f}".format(rand_for.score(X_test, y_test)))

Test set accuracy with Random Forest Classifier: 0.751


In [136]:
#effettuo la classificazione con logistic regression 
log_reg = LogisticRegression(max_iter=200)
risultato= log_reg.fit(X_train, y_train)
print("Test set accuracy with Logistic Regression: {:.3f}".format(risultato.score(X_test, y_test)))

Test set accuracy with Logistic Regression: 0.777


In [137]:
bnb = BernoulliNB()
bnb.fit(X_train, y_train)
print("Test set accuracy with Naive Bayes:{:.3f}".format(bnb.score(X_test,y_test)))

Test set accuracy with Naive Bayes:0.752


In [138]:
svm = SVC()
svm.fit(X_train, y_train)
print("Test set accuracy with Support Vector Machine:{:.3f}".format(svm.score(X_test,y_test)))

Test set accuracy with Support Vector Machine:0.728


In [139]:
dec_tree = DecisionTreeClassifier(random_state=1)
dec_tree.fit(X_train, y_train)
print("Test set accuracy with Decision Tree Classifier:{:.3f}".format(dec_tree.score(X_test, y_test)))

Test set accuracy with Decision Tree Classifier:0.634
